In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

In [2]:
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

In [3]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

CPU times: user 2min 29s, sys: 1min 35s, total: 4min 5s
Wall time: 4min 14s


In [4]:
%%time
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(0.0).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_train = all_train.drop(['date_time','srch_ci','srch_co','is_booking', 'cnt'], 1)

CPU times: user 8.71 s, sys: 29.3 s, total: 38 s
Wall time: 47.6 s


In [5]:
%%time
#Split into training and test data
#split = int(0.75*len(all_train))
#train = all_train[0:split]
#test  = all_train[split:]

CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 8.44 ms


In [5]:
%%time
features_train = all_train.ix[:,:'hotel_market'] 
labels_train = all_train.ix[:,'hotel_cluster':]
#features_test = test.ix[:,:'hotel_market'] 
#labels_test = test.ix[:,'hotel_cluster':]

CPU times: user 1.32 ms, sys: 3.63 ms, total: 4.95 ms
Wall time: 16.1 ms


In [6]:
features_train = features_train.values
labels_train = labels_train.values
#features_test = features_test.values
#labels_test = labels_test.values

In [7]:
%%time
clf = GaussianNB()
clf.fit(features_train, labels_train.ravel())

CPU times: user 46.6 s, sys: 47.1 s, total: 1min 33s
Wall time: 1min 56s


In [9]:
%%time
#pred = clf.predict(features_test)

CPU times: user 2min 9s, sys: 1min 34s, total: 3min 44s
Wall time: 6min 48s


In [11]:
#print accuracy_score(pred, labels_test.ravel())

0.0577647704175


In [8]:
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [9]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

CPU times: user 8.53 s, sys: 1.42 s, total: 9.96 s
Wall time: 10.4 s


In [10]:
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(0.0).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
testing_file = all_test.drop(['id', 'date_time','srch_ci','srch_co'], 1)

In [11]:
testing_file = testing_file.values

In [12]:
%%time
probs = pd.DataFrame(clf.predict_proba(testing_file))

CPU times: user 49.8 s, sys: 49.8 s, total: 1min 39s
Wall time: 1min 51s


In [14]:
probs = probs.transpose()

In [15]:
probs.head()

,0,1,2,3,4,5,6,7,8,9,...,2528233,2528234,2528235,2528236,2528237,2528238,2528239,2528240,2528241,2528242
0,1.506405e-10,2.955820e-11,6.870839e-02,0.002590,2.796015e-02,3.699272e-02,3.165751e-02,0.050388,0.003692,6.974000e-02,...,4.120701e-02,0.000011,0.001456,4.291106e-02,7.735057e-02,7.866100e-02,7.606394e-12,0.002261,4.141485e-02,2.020798e-08
1,0.000000e+00,0.000000e+00,3.084643e-09,0.425499,2.962716e-14,1.125613e-57,4.603395e-08,0.000000,0.422830,9.107173e-123,...,3.479091e-136,0.000000,0.000000,6.310056e-317,1.620888e-303,4.535872e-302,0.000000e+00,0.411110,3.055774e-135,0.000000e+00
2,1.716470e-02,3.227898e-03,8.642124e-03,0.000117,3.157087e-03,2.539271e-03,1.805579e-03,0.004502,0.000201,4.951708e-03,...,3.580276e-03,0.005322,0.010197,2.609661e-03,6.658094e-03,6.948049e-03,1.716767e-03,0.000173,2.433319e-03,2.663613e-03
3,6.431745e-03,1.322360e-02,3.644911e-03,0.000041,8.248447e-04,6.969966e-04,6.551082e-04,0.001014,0.000066,1.219071e-03,...,1.043317e-03,0.017990,0.008009,1.775961e-03,1.700150e-03,1.774696e-03,1.639449e-02,0.000053,1.408340e-03,3.094246e-02
4,2.356170e-04,2.389454e-05,6.770841e-03,0.000550,1.085496e-02,1.094781e-02,9.646123e-03,0.010380,0.000620,9.632881e-03,...,1.178748e-02,0.003144,0.006128,9.441841e-03,8.587547e-03,8.480219e-03,3.402525e-05,0.000594,1.060822e-02,1.872846e-03


In [16]:
%%time
#pred = clf.predict(testing_file)

CPU times: user 36 s, sys: 22.7 s, total: 58.6 s
Wall time: 1min 14s


In [18]:
submission = pd.DataFrame()

In [19]:
submission['id'] = pd.Series(all_test['id'])
submission['hotel_cluster'] = pd.Series(pred)

In [20]:
submission.to_csv('submission.csv', index=False)